For more on black library and code formating see https://medium.com/openplanetary/code-formatting-in-jupyter-cells-8fee4eda072f

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# pip install confluent-kafka

<IPython.core.display.Javascript object>

In [3]:
from configparser import ConfigParser
from confluent_kafka import Producer
import socket

<IPython.core.display.Javascript object>

In [4]:
parser = ConfigParser()
_ = parser.read("notebook.cfg")
kakfaUsername = parser.get("my_Notebook", "sasl.username")
kakfaPassword = parser.get("my_Notebook", "sasl.password")
conf = {
    "bootstrap.servers": "pkc-419q3.us-east4.gcp.confluent.cloud:9092",
    "security.protocol": "SASL_SSL",
    "sasl.mechanism": "PLAIN",
    "sasl.username": kakfaUsername,
    "sasl.password": kakfaPassword,
    "client.id": socket.gethostname(),
}

producer = Producer(conf)
topics = producer.list_topics()
print(topics)

lkc-7pxqnj


<IPython.core.display.Javascript object>

Settings

In [5]:
topicName = "speedtest"
sleepMinutes = 20

<IPython.core.display.Javascript object>

Ensure speedtest-cli is installed in your environment

In [6]:
# pip install speedtest-cli

<IPython.core.display.Javascript object>

Imports

In [7]:
import os
import speedtest
import socket
import requests
import json
import datetime
import time
import calendar

<IPython.core.display.Javascript object>

Define function to generate speed test results

In [8]:
def test():
    s = speedtest.Speedtest()
    ip = socket.gethostbyname(socket.gethostname())
    ipUrl = "https://api.ipify.org/?format=json"
    response = json.loads(requests.get(ipUrl).text)
    print(response)
    print(response["ip"])
    externalIp = response["ip"]
    s.get_servers()
    s.get_best_server()
    s.download()
    s.upload()
    res = s.results.dict()
    return res["download"], res["upload"], res["ping"], externalIp

<IPython.core.display.Javascript object>

Define function to send data to kafka server

In [9]:
def acked(err, msg):
    print("Massage Acked")
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))


def SendToStream(data):
    try:
        print("enter SendToStream", data)
        result = producer.produce(topicName, key="reading", value=data, callback=acked)
        producer.flush()
        print("Sent data")
    except Exception as e:
        print("Unable to send data", e)

<IPython.core.display.Javascript object>

Helper function to return timestamp

In [10]:
def GetTimestamp():
    gmTime = time.gmtime()
    now = calendar.timegm(gmTime)
    print("GetTimestamp", now, gmTime)
    return now

<IPython.core.display.Javascript object>

Main logic - get timesamp, run test, send results to kafka, sleep and repeat

In [ ]:
print("ready to start work")
linesToWrite = []
while True:
    try:
        now = GetTimestamp()
        d, u, p, ip = test()
        print(d, u, p, ip)
    except Exception as ex:
        print("exception", ex)
        d = -1
        u = -1
        p = -1
        ip = socket.gethostbyname(socket.gethostname())
    eventData = json.dumps(
        {"utc": now, "ip": ip, "download": d, "upload": u, "ping": p}
    )
    SendToStream(eventData)
    if sleepMinutes > 0:
        print("sleeping", sleepMinutes)
        time.sleep(sleepMinutes * 10)  # time.sleep takes seconds as parameter

ready to start work
GetTimestamp 1694300070 time.struct_time(tm_year=2023, tm_mon=9, tm_mday=9, tm_hour=22, tm_min=54, tm_sec=30, tm_wday=5, tm_yday=252, tm_isdst=0)
{'ip': '98.97.18.13'}
98.97.18.13
6495605.582977719 5297714.570500506 71.762 98.97.18.13
enter SendToStream {"utc": 1694300070, "ip": "98.97.18.13", "download": 6495605.582977719, "upload": 5297714.570500506, "ping": 71.762}
Massage Acked
Message produced: <cimpl.Message object at 0x0000019740A1E640>
Sent data
sleeping 20
GetTimestamp 1694300296 time.struct_time(tm_year=2023, tm_mon=9, tm_mday=9, tm_hour=22, tm_min=58, tm_sec=16, tm_wday=5, tm_yday=252, tm_isdst=0)
{'ip': '98.97.18.13'}
98.97.18.13
2112227.374288108 11715944.260317484 62.025 98.97.18.13
enter SendToStream {"utc": 1694300296, "ip": "98.97.18.13", "download": 2112227.374288108, "upload": 11715944.260317484, "ping": 62.025}
Massage Acked
Message produced: <cimpl.Message object at 0x0000019740A1E4C0>
Sent data
sleeping 20
GetTimestamp 1694300521 time.struct_ti